# Testing unitarity of SRG transformations in momentum distributions

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ February 9, 2023

The SRG transformations are unitary and therefore satisfy the condition<br>

$$
\begin{align}
U^\dagger U &= I + \frac{1}{4} \sum_{\mathbf{k}, \mathbf{k'}, \mathbf{K}} \bigl[
    \delta U(\mathbf{k},\mathbf{k'}) + \delta U^\dagger(\mathbf{k},\mathbf{k'})
    + \frac{1}{2} \sum_{\mathbf{k''}} \delta U(\mathbf{k},\mathbf{k''}) \delta U^\dagger(\mathbf{k''},\mathbf{k'})
\bigr]
a^\dagger_{\frac{\mathbf{K}}{2}+\mathbf{k}} a^\dagger_{\frac{\mathbf{K}}{2}-\mathbf{k}}
a_{\frac{\mathbf{K}}{2}-\mathbf{k'}} a_{\frac{\mathbf{K}}{2}+\mathbf{k'}} \\
&= I
.
\end{align}
$$

This implies that<br>

$$
\delta U(\mathbf{k},\mathbf{k'}) + \delta U^\dagger(\mathbf{k},\mathbf{k'}) 
+ \frac{1}{2} \sum_{\mathbf{k''}} \delta U(\mathbf{k},\mathbf{k''}) \delta U^\dagger(\mathbf{k''},\mathbf{k'})
= 0
.
$$

_Last update:_ February 9, 2023

In [1]:
# Python imports
import numpy as np
import numpy.linalg as la

In [2]:
# Imports from scripts
from scripts.integration import unattach_weights_from_matrix
from scripts.momentum_projection_operator import momentum_projection_operator
from scripts.potentials import Potential
from scripts.srg import get_transformation

In [3]:
import test_momentum_distribution_script as tmds

## Check partial wave basis first

In [4]:
kvnn = 6
channel = '3S1'
kmax, kmid, ntot = 15.0, 3.0, 120
generator, lamb = 'Wegner', 1.35

potential = Potential(kvnn, channel, kmax, kmid, ntot)
k_array, k_weights = potential.load_mesh()

In [5]:
H_initial = potential.load_hamiltonian()  # MeV
H_evolved = potential.load_hamiltonian('srg', generator, lamb)  # MeV
U_matrix = get_transformation(H_initial, H_evolved)  # Unitless
I_matrix = np.eye(len(U_matrix), len(U_matrix))

In [6]:
delta_U = U_matrix - I_matrix
delta_U_dag = (U_matrix - I_matrix).T

In [7]:
delta_U_3S1_3S1 = delta_U[:ntot, :ntot]
delta_U_dag_3S1_3S1 = delta_U_dag[:ntot, :ntot]
delta_U_3S1_3D1 = delta_U[:ntot, ntot:]
delta_U_dag_3D1_3S1 = delta_U_dag[ntot:, :ntot]

In [8]:
matrix = delta_U_3S1_3S1 + delta_U_dag_3S1_3S1 + 1/2 * (
    delta_U_3S1_3S1 @ delta_U_dag_3S1_3S1
    + delta_U_3S1_3D1 @ delta_U_dag_3D1_3S1
)
print(matrix)
print(f"Norm = {la.norm(matrix):.5f}")
# matrix_no_weights = unattach_weights_from_matrix(k_array, k_weights, matrix)
# print(matrix_no_weights)
# print(f"Norm = {la.norm(matrix_no_weights):.5f}")

[[-2.10389182e-01 -2.10275551e-01 -2.10207154e-01 ...  1.56883280e-05
   1.12674699e-05  1.11755458e-05]
 [-2.10275551e-01 -2.10282806e-01 -2.10226941e-01 ...  1.56881934e-05
   1.12677014e-05  1.11758152e-05]
 [-2.10207154e-01 -2.10226941e-01 -2.10213508e-01 ...  1.56877496e-05
   1.12695544e-05  1.11778520e-05]
 ...
 [ 1.56883280e-05  1.56881934e-05  1.56877496e-05 ... -1.95424400e-02
  -1.57116646e-02 -1.27308744e-02]
 [ 1.12674699e-05  1.12677014e-05  1.12695544e-05 ... -1.57116646e-02
  -2.46548914e-02 -2.26798500e-02]
 [ 1.11755458e-05  1.11758152e-05  1.11778520e-05 ... -1.27308744e-02
  -2.26798500e-02 -2.12059156e-02]]
Norm = 4.69298


## Check that normalization of momentum projection operator is conserved

In [9]:
q = 2.0
mpo_initial = momentum_projection_operator(q, k_array, k_weights, coupled=True)

print(f"Norm = {la.norm(mpo_initial)}")

mpo_I_matrix = I_matrix @ mpo_initial @ I_matrix
mpo_delU_matrix = delta_U @ mpo_initial @ I_matrix
mpo_delUdag_matrix = I_matrix @ mpo_initial @ delta_U_dag
mpo_delU2_matrix = delta_U @ mpo_initial @ delta_U_dag

mpo_evolved = (mpo_I_matrix + mpo_delU_matrix + mpo_delUdag_matrix
               + mpo_delU2_matrix)

print(f"Norm = {la.norm(mpo_evolved)}")

Norm = 2.847882658769522
Norm = 2.8478826587695085
